In [1]:
%pip install --upgrade llama-index        # core
%pip install --upgrade llama-index-llms-google-genai  # Google / Gemini LLM integration
%pip install --upgrade llama-index-embeddings-google-genai  # embeddings via Google GenAI
%pip install --upgrade google-generativeai  # underlying Google SDK
%pip install --upgrade pinecone-client  


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from llama_index.core import (
 SimpleDirectoryReader,
 Settings,
 SummaryIndex,
 TreeIndex,
 KeywordTableIndex,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding 

In [5]:
# <-- works after plugin install
# ============================ Embedding layer (no API calls)
#============================
# Small, fast local model (384-d). You can swap to"all-mpnet-base-v2" (768-d) if you prefer.
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
Settings.llm = None # hard-disable any LLM usage

LLM is explicitly disabled. Using MockLLM.


In [6]:
# ============================ Data layer (PDFs only)
#============================
try:
 docs = SimpleDirectoryReader(input_dir="./GenAI/GenAI-NoteBooks/coffee_pages",required_exts=[".html"]).load_data()
except TypeError:
 docs = SimpleDirectoryReader(input_dir="./GenAI/GenAI-NoteBooks/coffee_pages",file_exts=[".html"]).load_data()

In [7]:
# Build each index locally (no Pinecone, no fusion)
summary_idx = SummaryIndex.from_documents(docs) # precomputedper-doc summaries
keyword_idx = KeywordTableIndex.from_documents(docs) # exact/termlookups
tree_idx = TreeIndex.from_documents(docs) # hierarchicalzoom-in

2025-11-29 00:09:03,925 - INFO - > Building index from nodes: 3 chunks


In [9]:
# ============================ Query layer (retrieval-only; no LLM spend) ============================
def preview_top_nodes(index, query: str, k: int = 3):
 hits = index.as_retriever(similarity_top_k=k).retrieve(query)
 print(f"\nTop {k} for: {query}")
 for h in hits:
     print("-" * 70)
     print(h.node.get_content()[:700], "...")

preview_top_nodes(summary_idx, "High-level summary of these PDFs")
preview_top_nodes(keyword_idx, "ashwagandha in titles or headings")
preview_top_nodes(tree_idx, "brewing temperature guidance")

2025-11-29 00:24:07,977 - INFO - > Starting query: ashwagandha in titles or headings
2025-11-29 00:24:07,980 - INFO - query keywords: ['format', 'best', '10', 'ashwagandha', 'stopwords', 'separated', 'headings', 'comma', 'a question is provided below. given the question', "extract up to 10 keywords from the text. focus on extracting the keywords that we can use to best lookup answers to the question. avoid stopwords.\n---------------------\nashwagandha in titles or headings\n---------------------\nprovide keywords in the following comma-separated format: 'keywords: <keywords>'", 'question', 'following', 'extracting', 'extract', 'provided', 'focus', 'given', 'answers', 'lookup', 'titles', 'use', 'provide', 'keywords', 'text', 'avoid']
2025-11-29 00:24:07,981 - INFO - > Extracted keywords: ['format', 'best', '10', 'ashwagandha', 'stopwords', 'separated', 'comma', 'following', 'extract', 'provided', 'focus', 'given', 'use', 'provide', 'keywords', 'text', 'avoid']
2025-11-29 00:24:08,069 -


Top 3 for: High-level summary of these PDFs
----------------------------------------------------------------------
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8" />
  <title>Ashwagandha Coffee (Adaptogenic Latte)</title>
  <meta name="description" content="Ashwagandha Coffee (Adaptogenic Latte) — Indian-themed coffee knowledge base page for RAG projects." />
  <meta name="keywords" content="ashwagandha, withania somnifera, adaptogen, stress, latte, ayurveda, indian coffee, recipe, culture, benefits, risks, faq" />
  <meta name="author" content="RAG Corpus Generator" />
  <meta name="date" content="August 17, 2025" />
  <style>
    body { font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif; line-height: 1.6; max-width: 820px; margin: 2rem auto; padding: 0 1rem; }
   ...
----------------------------------------------------------------------
If using plant milks, choose barista blends for better foam and mouthfeel.</p>
</section>

<section>
  <h2>M